In [31]:
from ankipandas import Collection
import pandas as pd
import re
import simplejson
import copy
from networkx.readwrite import json_graph
import json

In [52]:
col = Collection("/Users/zhanlinchen/Library/Application Support/Anki2")

In [53]:
nonsuspended = col.cards[(col.cards['cqueue'] != 'suspended') & (col.cards['cdeck'].str.contains("AnKingStep 1Zanki"))]

In [54]:
notes = nonsuspended.merge_notes().reset_index()

In [55]:
revs = col.revs.reset_index()

In [56]:
notes = notes.merge(revs, how='left', left_on="cid", right_on="cid")

In [57]:
notes = notes[notes["ntags"].astype(str).str.contains("HighYield")] # only select high yield ones
notes = notes[notes["ntags"].astype(str).str.contains("B&B")] # only select high yield ones
notes = notes[notes["ntags"].astype(str).str.contains("FirstAid")] # only select high yield ones

In [58]:
notes = notes[notes["nflds"].astype(str).str.contains("mp3") == False]

In [59]:
def get_tags(line):
    new_tags = []
    for i in line["ntags"]:
        if ("FirstAid" in i):
            new_tags.extend(i.split("::"))
    new_tags = sorted(list(set(new_tags)))[2:]
    #new_tags.remove("#FirstAid").remove("#AK_Step1_v11")
    return new_tags

notes["ntags"] = notes.apply(get_tags, axis=1)

In [60]:
notes = notes.reset_index()

In [61]:
notes["ntags"].iloc[0]

['03_Levothyroxine_Liothyronine', '05_Pharm', '08_Endocrine', 'Levothyroxine']

In [62]:
import networkx as nx

In [63]:
G = nx.Graph()
s = set()
for index, row in notes.iterrows():
    if index % 2000 == 0:
        print(index, G.number_of_edges())
    new_node = row['nid']
    new_attributes = row['ntags']
    if new_node not in s:
        s.add(new_node)
        G.add_node(new_node, my_attributes=new_attributes)
        for node, attrs in G.nodes(data=True):
            if node != new_node:
                added = False
                for attr in attrs['my_attributes']:
                    if attr in new_attributes:
                        if added == False:
                            G.add_edge(node, new_node)
                            added = True
                        else:
                            break
                    

0 0
2000 806315
4000 2215822
6000 4336522
8000 6677153
10000 9797818
12000 11704973


In [64]:
G.number_of_nodes()

10071

In [65]:
G.number_of_edges()

13875487

In [66]:
sorted([len(x) for x in nx.connected_components(G)], reverse=True)[:10]

[10071]

In [67]:
tree = nx.algorithms.tree.mst.minimum_spanning_tree(G)

In [68]:
G = tree

In [69]:
for component in list(nx.connected_components(G)):
    if len(component)<5:
        for node in component:
            G.remove_node(node)

In [70]:
G.number_of_nodes()

10071

In [71]:
G.number_of_edges()

10070

In [72]:
j = json_graph.node_link_data(G)

In [73]:
jnew = copy.deepcopy(j)

In [74]:
for idx,n in enumerate(jnew['nodes']):
    note_orig = notes[notes["nid"] == n['id']]
    time = note_orig.iloc[0]['rid']
    note = note_orig.iloc[0]['nflds'][0].replace('\'', '')
    jnew['nodes'][idx].pop('my_attributes', None)
    jnew['nodes'][idx]['note'] = note
    jnew['nodes'][idx]['time'] = time

In [75]:
with open('out_clean.json', 'w') as outfile:
    d = simplejson.dumps(jnew, ignore_nan=True)
    outfile.write(d)